In [ ]:
import json
import networkx as nx
import numpy as np
import itertools

import matplotlib as mpl
import matplotlib.pyplot as plt


def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 0.5),
                    textcoords="offset points",
                    ha='center', va='bottom')

with open('data/commonFriends.json', 'r') as file:
    friends_links = json.load(file)

with open('data/myFriends.json', 'r') as file:
    friends_info = json.load(file)
    
friends_info = friends_info['response']['items']

In [ ]:
# sex and city:
sex = dict(map(lambda d: (d['id'], -d['sex']), friends_info))
city = dict(map(lambda d: (d['id'], d['city']['id']) 
                if 'city' in d.keys() else (-1, -1), friends_info))

if -1 in city:
    del city[-1]
    
# personal: political and religion
personal = dict(map(lambda d: (d['id'],d['personal']) 
                    if 'personal' in d.keys() else (-1, -1), friends_info))
if -1 in personal:
    del personal[-1]
political = dict([(k,v['political']) for k, v in personal.items() if 'political' in v.keys()])
religion = dict([(k,v['religion']) for k, v in personal.items() if 'religion' in v.keys()])

# schools and universities:
schools = dict(map(lambda d: (d['id'],d['schools']) 
                    if 'schools' in d.keys() else (-1, -1), friends_info))
if -1 in schools:
    del schools[-1]

prepared_schools_info = {}
for user_id,school_list in schools.items():
    if school_list:
        new_list = [int(school['id']) for school in school_list]
        prepared_schools_info[user_id]=new_list
        
        
        
universities = dict(map(lambda d: (d['id'],d['universities']) 
                    if 'universities' in d.keys() else (-1, -1), friends_info))
if -1 in universities:
    del universities[-1]
    
prepared_universities_info = {}
for user_id,universities_list in universities.items():
    if universities_list:
        new_list = [int(university['id']) for university in universities_list]
        prepared_universities_info[user_id]=new_list

# occupation:
info = dict(map(lambda d: (d['id'],d['occupation']['type']) 
                    if 'occupation' in d.keys() else (-1, -1), friends_info))
if -1 in info:
    del info[-1]

In [ ]:
links_array = []
links_array.append(city)
links_array.append(political)
links_array.append(religion)
links_array.append(info)

schools_and_universities_array = []
schools_and_universities_array.append(prepared_schools_info)
schools_and_universities_array.append(prepared_universities_info)

In [ ]:
def add_edge_or_update_weight(edge, G):
    if G.has_edge(edge[0], edge[1]):
        G[edge[0]][edge[1]]['weight'] += 1
    else:
        G.add_edge(edge[0], edge[1], weight=1)
        
G = nx.Graph()

for friend_ID, common_friend_IDs in friends_links.items():
    friend_ID = int(friend_ID)
    edges_list = [(friend_ID, common_friend_ID) 
                  for common_friend_ID in common_friend_IDs
                  if friend_ID<common_friend_ID]
    for arr in links_array:
        for edge in edges_list:
            if (edge[0] in arr) and (edge[1] in arr):
                if arr[edge[0]]==arr[edge[1]]:
                    add_edge_or_update_weight(edge, G)
                    
    for arr in schools_and_universities_array:
        for edge in edges_list:
            if (edge[0] in arr) and (edge[1] in arr):
                if set(arr[edge[0]]).intersection(arr[edge[1]]):
                    add_edge_or_update_weight(edge, G)
        


In [ ]:
nodes_without_links = set(sex)-set(G.nodes)
G.add_nodes_from(nodes_without_links)

low, *_, high = sorted(sex.values())
norm = mpl.colors.Normalize(vmin=low, vmax=high, clip=True)
mapper_sex = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.coolwarm)


plt.figure(figsize=(20,20))
pos = nx.spring_layout(G, k=0.4)


plt.figure(figsize=(15,15))
weights = [G[u][v]['weight'] for u,v in G.edges()]
nx.draw(G, pos, nodelist=sex,
        node_color=[mapper_sex.to_rgba(i) 
                    for i in sex.values()],
        width=weights*2, edge_color='blue',with_labels=False, node_size = 100)

edge_labels=dict([((u,v,),d['weight']) for u,v,d in G.edges(data=True)])
nx.draw_networkx_edge_labels(G,pos,edge_labels=edge_labels)

plt.savefig("graphs/multiple_graph.png")
plt.show()

